In [3]:
# Syntax error being tracked here: https://github.com/microsoft/vscode-jupyter/issues/6635
# If it's annoying, workaround here: https://stackoverflow.com/a/35597119/768439
%load_ext autoreload
%autoreload 2

# TODO: Use this to profile: https://stackoverflow.com/questions/45893768/how-do-i-find-out-what-parts-of-my-code-are-inefficient-in-python
%load_ext line_profiler

In [14]:
import sys
sys.path.append('../')

from src.serialization_lib import *
from src.data_types import *
from IPython.display import HTML, display, Markdown
from typing import List

import os
import pandas as pd
import statistics
import datetime
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 100)
pd.options.display.float_format = "{:,.2f}".format

In [15]:
INITIAL_PORTFOLIO_VALUE = 10000

BASE_METRIC = EvaluationMetric.EV_EBIT
TEST_METRIC = EvaluationMetric.P_B
STOCKS_UNIVERSE = StockUniverse.LARGE
PORTFOLIO_WEIGHT_STRATEGY = StockBasketWeightApproach.EQUAL_WEIGHTING

INITIAL_PORTFOLIO_VALUE = 10000
PORTFOLIO_SIZE = 30
REBALANCE_DAYS = 90

DATA_PROCESSED_BASE_PATH = '/Volumes/SDCard/TipBackTest/processed_data'

DEVELOPMENT = False
env = 'dev' if DEVELOPMENT else 'prod'

In [37]:
filename = get_feather_filename('df_res', BASE_METRIC, TEST_METRIC, REBALANCE_DAYS, PORTFOLIO_SIZE, STOCKS_UNIVERSE, env)
filename = os.path.join(DATA_PROCESSED_BASE_PATH, filename)
df_res = read_df_from_feather(filename)

# TODO: Need to fix the source code that generates these dataframes
df_res.rename(columns={'index': 'date'}, inplace=True)
df_res.set_index('date', inplace=True)

filename = get_feather_filename('df_debug', BASE_METRIC, TEST_METRIC, REBALANCE_DAYS, PORTFOLIO_SIZE, STOCKS_UNIVERSE, env)
filename = os.path.join(DATA_PROCESSED_BASE_PATH, filename)
df_debug = read_df_from_feather(filename)

# TODO: Need to fix the source code that generates these dataframes
df_debug.rename(columns={'index': 'date'}, inplace=True)
df_debug.set_index('date', inplace=True)

In [39]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
def color_per_cell(v):
    per = float(v if str(v)[-1] != '%' else str(v)[:-1])
    if per < -25:
        return "background-color: orangered"
    elif per >= -25 and per < 0:
        return "background-color: #ffcccb"
    elif per >= 0 and per < 25:
        return "background-color: #90ee90"
    elif per >= 25:
        return "background-color: green"

def display_two_decimal_places(val: float) -> str:
    return f'${val:,.2f}'

def display_date_only(d: datetime) -> str:
    return d.strftime('%Y-%m-%d')

def stock_rebalance_instances_to_df(rebalances_array: List[StockRebalanceInstance]) -> pd.DataFrame:
    vars = list(map(lambda f: f.name, dataclasses.fields(rebalances_array[0])))
    df = pd.DataFrame.from_records([[getattr(i, v) for v in vars] for i in rebalances_array], columns=['ticker', 'prev', 'curr'])
    df['ticker'] = df['ticker'].str.replace("'", "")
    return df

def per_stocks_up(rebalances_array: List[StockRebalanceInstance]) -> str:
    return f'{round(100 * sum([1 for reb_i in rebalances_array if (reb_i.curr_price > reb_i.prev_price)]) / len(rebalances_array), 2)}%'

def p_change(r: pd.Series) -> str:
    return f'{round(100 * (r.base_portfolio_curr_price - r.base_portfolio_prev_price ) / r.base_portfolio_prev_price, 2)}%'

def p_change2(r: pd.Series) -> str:
    return f'{round(100 * (r.curr - r.prev ) / r.prev, 2)}%'

def get_date_based_df(df_debug, date):
    df_date = df_debug.loc[date]
    stock_rebalance_instances = df_date['base_portfolio_per_ticker_data']
    df = stock_rebalance_instances_to_df(stock_rebalance_instances)
    tickers_closed = [str(s) for s in df_date['base_portfolio_tickers_closed']]
    df['up'] = df.apply(lambda r: r['curr'] > r['prev'], axis=1)
    df['per_change'] = df.apply(lambda r: p_change2(r), axis=1)
    df['did_close'] = df.apply(lambda r: "Closed" if str(r.ticker) in tickers_closed else "", axis=1)
    df = df.sort_values(by='per_change', key=lambda col: col.map(lambda v: float(v[:-1])))    
    summary_row = [
        'Summary / Total ',
        round(sum(df['prev']), 20),
        round(sum(df['curr']), 20),
        f"{round(100 * sum(df['up']) / len(df['up']))}%",
        f"{round(statistics.mean([float(p[:-1]) for p in df['per_change']]), 2)}%",
        ""
    ]
    df.loc[len(df.index)] = summary_row
    return df

df_debug['per_stock_up'] = df_debug.apply(lambda r: per_stocks_up(list(r.base_portfolio_per_ticker_data)), axis=1)
df_debug['portfolio_change'] = df_debug.apply(lambda r: p_change(r), axis=1)

df_debug_readable = df_debug.copy()
df_debug_readable['num_tickers_closed'] = df_debug_readable.apply(lambda r: len(r.base_portfolio_tickers_closed), axis=1)
df_debug_readable = df_debug_readable.drop(['base_portfolio_tickers_closed', 'base_portfolio_per_ticker_data', 'new_base_portfolio_per_ticker_data'], axis=1)
df_debug_readable.rename(columns={'base_portfolio_prev_price': 'prev', 'base_portfolio_curr_price': 'curr'}, inplace=True)

date_to_inspect = df_debug.index[7]
df_date = get_date_based_df(df_debug, date_to_inspect)

In [40]:
df_debug_with_style = df_debug_readable.round(2).style \
    .format({
        'curr': display_two_decimal_places,
        'prev': display_two_decimal_places,
        'date': display_date_only,
        'prev_date': display_date_only,
        'curr_date': display_date_only,    
    }).set_caption(
        f"Return for \n" \
        f" for {str(STOCKS_UNIVERSE)}\n" \
        f" for {str(BASE_METRIC)}" \
        f" for rebalance period: {str(REBALANCE_DAYS)}" \
        f" for portfolio size: {str(PORTFOLIO_SIZE)}" \
    ).set_table_styles([{
        'selector': 'caption',
        'props': [
            ('color', 'black'),
            ('font-size', '16px'),
            ('text-align', 'center'),        
        ]}])

HTML(df_debug_with_style.applymap(color_per_cell, subset=['per_stock_up', 'portfolio_change']).render())

,prev_date,curr_date,prev,curr,per_stock_up,portfolio_change,num_tickers_closed
date,,,,,,,
2011-04-01 00:00:00,2011-01-03,2011-04-01,"$10,000.00","$9,177.86",40.0%,-8.22%,0
2011-07-01 00:00:00,2011-04-01,2011-07-01,"$9,177.86","$7,511.72",33.33%,-18.15%,0
2011-09-30 00:00:00,2011-07-01,2011-09-30,"$7,511.72","$6,337.28",20.0%,-15.63%,1
2011-12-29 00:00:00,2011-09-30,2011-12-29,"$6,337.28","$6,076.20",46.67%,-4.12%,0
2012-03-28 00:00:00,2011-12-29,2012-03-28,"$6,076.20","$7,052.47",83.33%,16.07%,0
2012-06-26 00:00:00,2012-03-28,2012-06-26,"$7,052.47","$5,918.07",33.33%,-16.09%,0
2012-09-24 00:00:00,2012-06-26,2012-09-24,"$5,918.07","$6,320.85",60.0%,6.81%,0
2012-12-21 00:00:00,2012-09-24,2012-12-21,"$6,320.85","$5,641.95",30.0%,-10.74%,0
2013-03-22 00:00:00,2012-12-21,2013-03-22,"$5,641.95","$6,633.35",70.0%,17.57%,2


In [41]:
df_with_style = df_date.style \
    .format({
        'curr': display_two_decimal_places,
        'prev': display_two_decimal_places,
    }) \
    .set_caption(
        f"Rebalance results" \
        f" on {datetime.datetime.strftime(date_to_inspect, '%Y-%m-%d')}" \
        f" for {str(STOCKS_UNIVERSE)}" \
        f" for {str(BASE_METRIC)}") \
    .set_table_styles([{
        'selector': 'caption',
        'props': [
            ('color', 'black'),
            ('font-size', '16px'),
            ('text-align', 'center'),        
        ]
}])

HTML(df_with_style.applymap(color_per_cell, subset=['per_change']).render())

,ticker,prev,curr,up,per_change,did_close
15,LGHS,$1.09,$0.26,False,-76.15%,
7,CPHI,$0.37,$0.20,False,-45.95%,
1,BOPH,$0.85,$0.48,False,-43.53%,
8,CRTP,$0.70,$0.40,False,-42.86%,
4,CHBT,$2.90,$1.69,False,-41.72%,
27,VRNM,$3.45,$2.20,False,-36.23%,
18,MSN,$1.29,$1.06,False,-17.5%,
9,CVVT,$0.52,$0.43,False,-17.47%,
6,CMKG,$1.15,$0.97,False,-15.65%,
29,XUE,$2.58,$2.24,False,-13.07%,


In [42]:
def cagr(start: float, end: float, n: int) -> float:
    cagr = pow(end / start, 1 / n) - 1
    return cagr * 100

def value_growth(cagr: float, n: int) -> float:
    return 10000 * pow(1 + cagr / 100, n)    

def display_data(data: dict, caption: str) -> None:
    df = pd.DataFrame(data, columns = ['Time Period', 'CAGR', 'SD', '$10,000 grows to...'])
    df = df.style.set_caption(caption)\
        .format({"CAGR": "{:.2f}%", "SD": "{:.2f}%", '$10,000 grows to...': "${:,.2f}"}) \
        .set_table_styles([{
            'selector': 'caption',
            'props': [
                ('color', 'black'),
                ('font-size', '16px'),
                ('text-align', 'center'),        
            ]
        }])
    display(HTML(df.applymap(color_per_cell, subset=['CAGR', 'SD']).render()))

min_date = min(df_res.index)
max_date = max(df_res.index)

time_period_table_days = 365 * 1 # 1 year(s)

tabulated_base_data = []
tabulated_test_data = []

period_start_date = min_date

idx_s = 0
idx_e = 0

for date, data in df_res.iterrows():
    idx_e += 1
    td = (date - period_start_date)
    if (td.days >= time_period_table_days - 5) and idx_s < len(df_res):
        n = round(td.days / 365)    
        df = df_res.iloc[idx_s:idx_e]
        
        prev_data = df_res.iloc[idx_s]
        curr_data = df_res.iloc[idx_e]
        
        period_start_dateiod = list(d.strftime('%Y-%m-%d') for d in [period_start_date, date])
        
        base_start_p = prev_data['base_price']
        base_end_p = curr_data['base_price']
        base_cagr = cagr(base_end_p, base_start_p, n)
        base_sd = np.std(df['base_price'].values)
        base_sd = base_sd / np.sum(df['base_price'].values) * 100
        base_growth = value_growth(base_cagr, n)
        
        tabulated_base_data.append([period_start_dateiod, base_cagr, base_sd, base_growth])
        
        test_start_p = prev_data['test_price']
        test_end_p = curr_data['test_price']
        test_cagr = cagr(test_start_p, test_end_p, n)
        test_sd = np.std(df['test_price'].values)
        test_sd = test_sd / np.sum(df['test_price'].values) * 100
        test_growth = value_growth(test_cagr, n)

        tabulated_test_data.append([period_start_dateiod, test_cagr, test_sd, test_growth])
        
        idx_s = idx_e + 1
        idx_e += 1
        period_start_date = date
        

display_data(tabulated_base_data, f"{BASE_METRIC}")
display_data(tabulated_test_data, f"{TEST_METRIC}")

,Time Period,CAGR,SD,"$10,000 grows to..."
0,"['2011-01-03', '2011-12-29']",41.79%,3.95%,"$14,179.43"
1,"['2011-12-29', '2013-03-22']",-25.62%,1.84%,"$7,438.17"
2,"['2013-03-22', '2014-03-18']",-4.32%,1.34%,"$9,568.13"
3,"['2014-03-18', '2015-03-13']",-1.60%,0.20%,"$9,839.61"
4,"['2015-03-13', '2016-03-07']",-14.81%,2.09%,"$8,519.34"
5,"['2016-03-07', '2017-03-02']",-21.27%,2.50%,"$7,872.96"
6,"['2017-03-02', '2018-05-25']",9.21%,1.92%,"$10,921.00"
7,"['2018-05-25', '2019-05-21']",-57.26%,6.96%,"$4,274.07"


,Time Period,CAGR,SD,"$10,000 grows to..."
0,"['2011-01-03', '2011-12-29']",-28.87%,4.58%,"$7,113.28"
1,"['2011-12-29', '2013-03-22']",21.47%,1.32%,"$12,146.53"
2,"['2013-03-22', '2014-03-18']",-25.47%,2.37%,"$7,452.55"
3,"['2014-03-18', '2015-03-13']",-32.70%,5.09%,"$6,730.17"
4,"['2015-03-13', '2016-03-07']",-14.19%,1.04%,"$8,581.25"
5,"['2016-03-07', '2017-03-02']",-10.18%,1.05%,"$8,982.02"
6,"['2017-03-02', '2018-05-25']",-28.08%,3.91%,"$7,192.24"
7,"['2018-05-25', '2019-05-21']",389.09%,14.49%,"$48,909.38"
